In [14]:
import pandas as pd
import geopandas as gpd
import fiona
import requests
import zipfile
import io
import glob
import os
from owslib.wfs import WebFeatureService
from fiona import BytesCollection
#from arcgis.features import FeatureLayerCollection

ModuleNotFoundError: No module named 'arcgis'

In [68]:
fossil_fuel_urls = {'no': 'https://factpages.npd.no/downloads/fgdb/NPD_FactMapsData_v3_0.zip',
                    'uk': 'https://datanstauthority.blob.core.windows.net/external/OpenDataZips/UKCS_OFF_ED50.zip',
                    'int': 'https://ows.emodnet-humanactivities.eu/wfs?SERVICE%3DWFS&REQUEST%3DGetCapabilities&VERSION=2.0.0'}

In [86]:
import datetime
from dateutil.relativedelta import relativedelta

today = datetime.date.today() - relativedelta(months=1)

year = today.year
month_name = today.strftime('%b').lower()
month_number = today.strftime('%m')

In [ ]:
nl_infra = f'https://www.nlog.nl/sites/default/files/{year}-{month_number}/{month_name}-{year}-nlog-facility_utm.zip'
nl_licences = f'https://www.nlog.nl/sites/default/files/{year}-{month}/{month}-{year}-nlog-licences_utm.zip'
nl_fields = f'https://www.nlog.nl/sites/default/files/{year}-{month}/{month}-{year}-nlog-fields_utm.zip'
nl_wellbores = f'https://www.nlog.nl/sites/default/files/{year}-{month}/{month}-{year}-nlog-boreholes_utm.zip'
nl_pipes = 'https://geo.rijkswaterstaat.nl/services/ogc/gdr/kabels_en_leidingen_noordzee/ows?'

'apr'

'04'

In [41]:
url = 'https://www.gdngeoservices.nl/arcgis/services/nlog/gdw_ng_licence_utm/MapServer/WFSServer?'

In [42]:
def get_wfs_layers(url):
    '''
    Get list of available layers
    and their index
    '''
    
    wfs = WebFeatureService(url, version='2.0.0')
    
    for i, layer in enumerate(wfs.contents):
        print(i, layer)

In [43]:
get_wfs_layers(url)

0 gdw_ng_licence_utm:NLOG.GDW_NG_LICENCE_UTM


In [55]:
wfs = WebFeatureService(url, version='2.0.0')

In [64]:
layer_name = list(wfs.contents)[-1]

In [66]:
from requests import Request

params = dict(service='WFS', version="2.0.0", request='GetFeature',
      typeName=layer_name, outputFormat='json', count=100, startIndex=0)

# Parse the URL with parameters
wfs_request_url = Request('GET', url, params=params).prepare().url

# Read data from URL
data = gpd.read_file(wfs_request_url, format='GML')

DriverError: '/vsimem/56b7924944644783b577024821858371' not recognized as a supported file format.

In [44]:
def select_wfs_layer(url, index):
    '''
    Select a layer by index
    '''
    wfs = WebFeatureService(url, version='2.0.0')
    layer = list(wfs.contents)[index]
    
    return layer

In [45]:
def wfs2gdf(layer, url, output_format, wfs_version="2.0.0"):
    '''
    Needs layer, wfs_url and output_format
    as input and creates a geodataframe
    '''
    
    params = dict(service='WFS', 
                  version=wfs_version, 
                  request='GetFeature', 
                  typeName=layer, 
                  outputFormat=output_format)
    
    if 'xml' in output_format:
        with BytesCollection(requests.get(url, params=params).content) as f:
            gdf = gpd.GeoDataFrame.from_features(f)
    
    elif 'json' in output_format:
            r = requests.get(url, params=params)
            gdf = gpd.read_file(r.text)
        
    return gdf

In [54]:
gpd.read_file(url)

HTTPError: HTTP Error 400: 

In [52]:

wfs2gdf(select_wfs_layer(url, 0), url, 'xml')

ERROR 4: `/vsimem/3d5b4926504441cd97652631d848a470' not recognized as a supported file format.


DriverError: '/vsimem/3d5b4926504441cd97652631d848a470' not recognized as a supported file format.

In [11]:
def get_arcgis_layers(url):
    '''
    Get layers from ArcGIS 
    FeatureServer
    '''
    arcgis_layers = FeatureLayerCollection(url)
    
    for i, layer in enumerate(arcgis_layers.layers):
        print(i, layer.properties.name)
    
def arcgis2gdf(url, layer_index):
    '''
    Use layer index to import
    data from FeatureServer in gdf
    '''
    
    collection = FeatureLayerCollection(url)
    layer = collection.layers[layer_index]
    
    data = layer.query()
    
    geojson_string = data.to_geojson
    geojson_dict = json.loads(geojson_string)
    
    gdf = gpd.GeoDataFrame.from_features(geojson_dict['features'])
    
    return gdf

In [12]:
url = fossil_fuel_urls.get('nl').get('fields')

In [13]:
get_arcgis_layers(url)

NameError: name 'FeatureLayerCollection' is not defined